In [ ]:
from scipy.optimize import least_squares
import numpy as np
import matplotlib.pyplot as plt

def parratt_1film(alpha_deg, lambda_A, params):

    d_A, delta_f, beta_f, delta_s, beta_s, sigma01, sigma12, bg, alpha0_deg, logS = params

    # offset
    a = np.deg2rad(alpha_deg + alpha0_deg)
    k0 = 2*np.pi / lambda_A

    n0 = 1.0 + 0.0j
    n1 = (1.0 - delta_f) + 1j*beta_f
    n2 = (1.0 - delta_s) + 1j*beta_s

    kz0 = k0 * np.sqrt((n0**2 - np.cos(a)**2) + 0j)
    kz1 = k0 * np.sqrt((n1**2 - np.cos(a)**2) + 0j)
    kz2 = k0 * np.sqrt((n2**2 - np.cos(a)**2) + 0j)

    r01 = (kz0 - kz1) / (kz0 + kz1)
    r12 = (kz1 - kz2) / (kz1 + kz2)

    # nevot croce
    r01 *= np.exp(-2 * kz0 * kz1 * sigma01**2)
    r12 *= np.exp(-2 * kz1 * kz2 * sigma12**2)

    phase = np.exp(2j * kz1 * d_A)

    R2 = 0.0 + 0.0j
    R1 = (r12 + R2 * phase) / (1 + r12 * R2 * phase)
    R0 = (r01 + R1 * phase) / (1 + r01 * R1 * phase)

    R = np.abs(R0)**2

    S = np.exp(logS)   
    return S * R + bg


def make_residuals_log10(alpha_deg, R_meas, lambda_A,
                         delta_s_fixed, beta_s_fixed, bg_fixed):
    
    R_meas = np.asarray(R_meas)

    def residuals(p_fit):
        d_A, delta_f, beta_f, sigma01, sigma12, alpha0_deg, logS = p_fit
        p_full = (d_A, delta_f, beta_f,
                  delta_s_fixed, beta_s_fixed,
                  sigma01, sigma12,
                  bg_fixed, alpha0_deg, logS)

        Rm = parratt_1film(alpha_deg, lambda_A, p_full)

        if not np.all(np.isfinite(Rm)):
            return np.ones_like(R_meas) * 1e6

        Rm = np.clip(Rm, 1e-14, None)
        Rm_meas = np.clip(R_meas, 1e-14, None)
        return np.log10(Rm) - np.log10(Rm_meas)

    return residuals



delta_s_fixed = 7.6e-6
beta_s_fixed  = 1.9e-7
bg_fixed      = 1e-9   

d_nm_guess = 88.6
d_A_guess = d_nm_guess * 10.0

p0_fit = np.array([
    d_A_guess,  # d_A 
    1.6e-6,     # delta_f
    1.7e-8,     # beta_f
    16.0,       # sigma01 
    9.6,       # sigma12 
    0.0,        # alpha0_deg
    0.0         
], dtype=float)

# ---- Bounds ----
lb_fit = np.array([
    700.0,    # d_A
    1e-6,    # delta_f
    0.0,     # beta_f
    0.5,     # sigma01
    0.5,     # sigma12
    -0.05,   # alpha0_deg 
    np.log(0.2)   
], dtype=float)

ub_fit = np.array([
    1000.0,  # d_A
    6e-6,    # delta_f
    1.8e-7,    # beta_f
    30.0,    # sigma01
    30.0,    # sigma12
    +0.05,   # alpha0_deg
    np.log(5.0)   
], dtype=float)

residual_fun = make_residuals_log10(a_fit, R_fit, lambda_A,
                                    delta_s_fixed, beta_s_fixed, bg_fixed)

res = least_squares(
    residual_fun,
    p0_fit,
    bounds=(lb_fit, ub_fit),
    loss="soft_l1",
    f_scale=0.05,
    x_scale="jac"
)

popt_fit = res.x
d_A, delta_f, beta_f, sigma01, sigma12, alpha0_deg, logS = popt_fit
S = np.exp(logS)

popt_full = np.array([d_A, delta_f, beta_f, delta_s_fixed, beta_s_fixed,
                      sigma01, sigma12, bg_fixed, alpha0_deg, logS], dtype=float)

R_model = parratt_1film(a_fit, lambda_A, popt_full)

plt.figure()
plt.semilogy(a_fit, np.clip(R_fit, 1e-14, None), ".", label="Messdaten")
plt.semilogy(a_fit, np.clip(R_model, 1e-14, None), "-", label="Parratt-Fit")
plt.xlabel(r"$\alpha\,/\circ$")
plt.ylabel("Reflektivität")
plt.legend()
plt.tight_layout()
plt.savefig("plots/parratt.pdf")
plt.show()

alpha_c_poly = np.degrees(np.sqrt(2*delta_f))
alpha_c_si = np.degrees(np.sqrt(2*delta_s_fixed))

print(f"d = {d_A/10:.2f} nm")
print(f"delta_f (film) = {delta_f:.3e}")
print(f"delta_s (Si)   = {delta_s_fixed:.3e}  (fixed)")
print(f"sigma01 (air/film) = {sigma01:.2f} Å")
print(f"sigma12 (film/Si)  = {sigma12:.2f} Å")
print(f"beta_f (film) = {beta_f:.3e}")
print(f"beta_s (Si)   = {beta_s_fixed:.3e}  (fixed)")
print(f"bg   = {bg_fixed:.3e}  (fixed)")
print(f"alpha0 = {alpha0_deg:+.4f} deg")
print(f'alpha_c_poly = {alpha_c_poly:.3f}')
print(f'alpha_c_si = {alpha_c_si:.3f}')
print(f"S = {S:.4f}")
